<a href="https://colab.research.google.com/github/SaswatD27/21cs460_group01/blob/main/Code/CS460_FL_1_02_Functional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes
* Current client side implementation just helps create initial model; no server2client communication yet, no preexisting initial model. Is this necessary? Don't think so, not for now.
* Will Noising before Aggregation FL work? Let us see. Perhaps the exponential mechanism will be better. (Implemented, works, hmm... what's the error margin now?)
* Need to add error/loss function graph
* Need to decouple n_k from training functions to apply some algorithms and make it more natural - Update: Done
* IMPORTANT:  Need testing set (might not but still, for propriety; client-side) - Update: Done
* Gaussian noise addition (Client side) works but might be identifying, implement mixes/LDP/random subsampling of returned weights?

## Bootstrapping

In [ ]:
import random
import math
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from copy import deepcopy

## Loading MNIST

In [ ]:
#(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
#emnist_train, emnist_test = datasets.emnist.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
img_rows, img_cols = 28, 28

In [ ]:
print(train_labels)

### Partitioning MNIST (indices) for clients

In [ ]:
K=100
n_k= np.random.randint(100,600,K)
n_tot=sum(n_k)
client_images=deepcopy(list(range(len(train_images))))
random.shuffle(client_images)
x_train=[]
y_train=[]
'''
for i in range(K):
  x_train.append([])
  y_train.append([])
  for j in range(n_k[i]):
    k=client_images.pop()
    x_train[i].append(train_images[k])
    y_train[i].append(train_labels[k])
'''
indices_dev=[]
for i in range(K):
  indices_dev.append([])
  for j in range(n_k[i]):
    k=client_images.pop()
    indices_dev[i].append(k)  

In [ ]:
print(type(np.array(x_train,dtype=object)))

In [ ]:
print(n_tot)

## Defining DNN

In [ ]:
#Creating DNN using tensorflow
def create_DNN():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=32, activation='relu')) 
  #model.add(tf.keras.layers.Dropout(rate=0.2))
  #model.add(tf.keras.layers.Dense(units=512))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=10, activation='softmax')) 
  #model.add(tf.keras.layers.BatchNormalization())
  return model

##0. Centralised Learning

In [ ]:
model=create_DNN()
model.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
model.summary()
print(np.array(model.get_weights()))

In [ ]:
#Training Model
#history = model.fit(train_images[indices_dev[0]], train_labels[indices_dev[0]], epochs=25, validation_data=(test_images, test_labels))
history = model.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels))

In [ ]:
#Plotting Accuracy Graph
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
print(test_acc)

In [ ]:
print((model.get_weights()))

## Client Side Setup

In [ ]:
#Conducts local training and outputs weight vector
def device_local_learning(server_model,id): 
    model=server_model
    model.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    #Training the model using own data
    history = model.fit(train_images[indices_dev[id]],train_labels[indices_dev[id]], epochs=20, validation_data=(test_images, test_labels), verbose=0)
    
    return model, n_k[id]

## Server Side
### 1. FedAvg

In [ ]:
#Server making intial model
server_model=create_DNN()

In [ ]:
#Averages weights as part of FedAveraging
def avg_weights(client_frac,client_models,n_clients_round):
  client_models=np.array(client_models)
  res=client_frac[0]*np.array(client_models[0].get_weights())
  #print(res)
  for i in range(1,n_clients_round):
    res+=client_frac[i]*np.array(client_models[i].get_weights())
  return res

In [ ]:
#Federated Averaging: Tensorflow/Keras Sequential compatible
def fed_avg_DNN(server_model, client_models, client_n):
  client_frac=np.zeros(len(client_n))
  for i in range(len(client_n)):
    client_frac[i]=(client_n[i])*(1/np.sum(client_n))
  server_model.set_weights(avg_weights(client_frac,client_models,len(client_frac)))
  return server_model

In [ ]:
#Conducts the entire process of FL with vanilla FedAveraging
def fed_learning(T, K, C, server_model):
  client_models=[]
  client_n=[]
  test_err=[]
  z=0
  for i in range(T):
    training_subset = np.random.randint(0, K, int(C * K))
    #z=0
    for j in training_subset:
      w,n_c=device_local_learning(server_model,j)
      client_models.append(w)
      client_n.append(n_c)
      z+=1
  
    model_fin = fed_avg_DNN(server_model, client_models[:z], client_n[:z])#
    model_fin.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    test_err.append(model_fin.evaluate(test_images,  test_labels, verbose=0))
    print(i," done.","\n size =",len(client_models)," Accuracy : ",test_err[-1][1])
    
  return model_fin, test_err

In [ ]:
#Defining Parameters and Running the abovementioned functions
T = 20  # No of federated learning rounds
K = 100  # Total no of nodes
#C = 0.7  # fraction of nodes used at each iteration
C=0.25
# B = #Local batch size used at each learning iteration
# random Model
#w = np.random.rand(7)
#n = 100  # number of iteration for local training before pooling
#alp_ = 0.01  # local learning rate
#m = 5

time1 = time.time()
model_fin, test_err = fed_learning(T, K, C, server_model) #test_err = list of (loss, test error) tuples
time2 = time.time()
print("Time taken for SGD federated learning", time2 - time1)#

In [ ]:
#OPTIONAL: Very verbose, lots of numbers
print(model_fin.get_weights())

In [ ]:
#Not essential to run
model_fin.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])

In [ ]:
test_loss, test_acc = model_fin.evaluate(test_images,  test_labels, verbose=2)
model_fin.evaluate(train_images, train_labels, verbose=1)

In [ ]:
print(test_err)

In [ ]:
print(test_acc)

In [ ]:
# MUST RUN FOR CREATING PLOT
test_err_acc = [a[1] for a in test_err]

In [ ]:
#Creates Error Plot
number=np.arange(1,len(test_err_acc)+1)
plt.plot(number, test_err_acc, color='Blue')
plt.xlabel('No. of Communication Rounds')
plt.ylabel('Accuracy')
plt.title('DNN on MNIST : Accuracy w.r.t No. of Communication Rounds')
#plt.tight_layout()
plt.show()

In [ ]:
print(test_err)

## 1. Dynamic Sampling

In [ ]:
#Conducts FL with dynamic sampling
server_model_dyn=server_model
def fed_learning_dyn_samp(T, K, C, server_model, beta):
  client_models=[]
  client_n=[]
  test_err=[]
  z=0
  for i in range(T):
    c=C*(1/math.exp(beta*i)) #Sampling Rate Downscaled
    training_subset = np.random.randint(0, K, max(1,int(c * K)))
    #z=0
    for j in training_subset:
      w,n_c=device_local_learning(server_model,j)
      client_models.append(w)
      client_n.append(n_c)
      #print(z)
      z+=1
    #for i in range(len(client_models)):
    #  test_loss,test_acc=client_models[i].evaluate(test_images,  test_labels, verbose=2)
    #  print(test_acc)
    model_fin = fed_avg_DNN(server_model, client_models[:z], client_n[:z])#
    model_fin.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    test_err.append(model_fin.evaluate(test_images,  test_labels, verbose=0))
    print(i," done.","\n size =",len(client_models)," Accuracy : ",test_err[-1][1])
    #for i in range(len(client_models)):
    #  print("\n Client - ", i, " sends \n",client_models[i].get_weights())
  return model_fin, test_err

In [ ]:
#Runs the dynamic sampling model
T = 20  # No of federated learning rounds
K = 100  # Total no of nodes
#C = 0.7  # fraction of nodes used at each iteration
C=0.25
beta = 0.05

time1 = time.time()
model_fin, test_err = fed_learning_dyn_samp(T, K, C, server_model_dyn, beta) #test_err = list of (loss, test error) tuples
time2 = time.time()
print("Time taken for SGD federated learning", time2 - time1)#

In [ ]:
print(model_fin.get_weights())

In [ ]:
test_loss, test_acc = model_fin.evaluate(test_images,  test_labels, verbose=2)
model_fin.evaluate(train_images, train_labels, verbose=1)

In [ ]:
print(test_err)

In [ ]:
#MUST RUN TO GET PLOT
test_err_acc = [a[1] for a in test_err]

In [ ]:
#Code to Plot, run this
number=np.arange(1,len(test_err_acc)+1)
plt.plot(number, test_err_acc, color='Blue')
plt.xlabel('No. of Communication Rounds')
plt.ylabel('Accuracy')
plt.title('DNN on MNIST : Accuracy w.r.t No. of Communication Rounds')
#plt.tight_layout()
plt.show()

## 1*. Adaptive Sampling

In [ ]:
#Creates initial server model for the very first run (after start of runtime)
server_model=create_DNN()

In [ ]:
#Runs the adaptive sampling protocol
def fed_learning_adap_samp(T, K, C, server_model, beta, gamma):
  client_models=[]
  client_n=[]
  test_acc=[]
  error=[]
  z=0
  i0=0
  error_t=0
  beta0=beta
  for i in range(T):
    c=C*(1/math.exp(beta0*i0)) #Decay of sampling rate
    training_subset = np.random.randint(0, K, max(1,int(c * K)))
    #z=0
    for j in training_subset:
      w,n_c=device_local_learning(server_model,j)
      client_models.append(w)
      client_n.append(n_c)
      #print(z)
      z+=1
    
    model_fin = fed_avg_DNN(server_model, client_models[:z], client_n[:z])#
    model_fin.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    test_acc.append(model_fin.evaluate(test_images,  test_labels, verbose=0))
    error.append((1-test_acc[-1][1]))
    '''
    if i>0:
      if error[i]>error[i-1]:
        mult=((error[i-1]/error[i])**2)*(1/gamma)#*(i/i+1))
        print("Multiplier = ",mult)
        beta0*=mult
        print("New Decay Constant = ",beta0)
        if error_t==0:
          error_t=error[i-1]
        else:
          i0=i+1
          if error[i]<=error_t:
            beta0=beta
            error_t=0
            print("Beta = ",beta," restored!")
    else:
      i0=1
    print(i+1," done. Error = ", err_r)
    '''
    if i>0:
      if test_acc[i][1]<test_acc[i-1][1]: #Error increase detection
        multiplier=((1-test_acc[i-1][1])/(1-test_acc[i][1])) #If yes, then penalise w.r.t. the currently observed and prev round's error 
        beta0*=multiplier*gamma #Penalisation of decay coefficient
        if error_t==0:
          error_t=error[i-1] #Record error threshold to reattain if error threshold does not exist
        print(" Multiplier  = ", multiplier,"\n New Decay Constant = ",beta0)
      else:
        i0=i+1 #All good, can increase counter in sampling rate decay formula
        if error[i]<=error_t: #If error level falls below the threshold, restore the decay coeff!
          beta0=beta
          error_t=0
          print("Beta = ",beta," restored!")
    else:
      i0=1
    #for i in range(len(client_models)):
    #  print("\n Client - ", i, " sends \n",client_models[i].get_weights())
    print(i," done.","\n size =",len(client_models)," Accuracy : ",test_acc[-1][1]," Decay Coefficient : ",beta0)
  return model_fin, test_acc

In [ ]:
#Runs the adaptive sampling model
T = 20  # No of federated learning rounds
K = 100  # Total no of nodes
#C = 0.7  # fraction of nodes used at each iteration
C=0.25 #initial sampling rate
beta = 0.05
gamma = 1

time1 = time.time()
model_fin_adap, test_err_adap = fed_learning_adap_samp(T, K, C, server_model, beta, gamma) #test_err = list of (loss, test error) tuples
time2 = time.time()
print("Time taken for SGD federated learning", time2 - time1)#

In [ ]:
#Optional: Very verbose
print(model_fin_adap.get_weights())

In [ ]:
#Not necessary to run, works just fine (probably redundant code)
model_fin_adap.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])

In [ ]:
test_loss, test_acc = model_fin_adap.evaluate(test_images,  test_labels, verbose=2)
model_fin_adap.evaluate(train_images, train_labels, verbose=1)

In [ ]:
print(test_err_adap)

In [ ]:
print(test_acc)

In [ ]:
#MUST RUN TO GET PLOT
test_err_acc = [a[1] for a in test_err_adap]

In [ ]:
#Run to get error plot
number=np.arange(1,len(test_err_acc)+1)
plt.plot(number, test_err_acc, color='Blue')
plt.xlabel('No. of Communication Rounds')
plt.ylabel('Accuracy')
plt.title('DNN on MNIST with Adaptive Sampling : Accuracy w.r.t No. of Communication Rounds')
#plt.tight_layout()
plt.show()